# Belavkin Optimizer Analysis

This notebook demonstrates how to analyze results from Belavkin optimizer experiments.

In [ ]:
import sys
sys.path.insert(0, '..')

import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from belavkin_ml.utils.visualization import (
    plot_training_curves,
    plot_optimizer_comparison,
    plot_convergence_analysis,
)

sns.set_style('whitegrid')
%matplotlib inline

## Load Results

Load experimental results from JSON files.

In [ ]:
# Load results from modular arithmetic experiment
results_path = Path('../experiments/track1/results/modular_arithmetic/results.json')

if results_path.exists():
    with open(results_path, 'r') as f:
        results = json.load(f)
    print(f"Loaded results for: {list(results.keys())}")
else:
    print(f"Results not found. Run experiments first.")
    results = None

## Visualize Training Curves

In [ ]:
if results:
    # Plot comparison
    fig = plot_optimizer_comparison(
        all_results=results,
        metric='test_accs',
        title='Test Accuracy - Modular Arithmetic',
    )
    plt.show()

## Convergence Analysis

In [ ]:
if results:
    fig = plot_convergence_analysis(all_results=results)
    plt.show()

## Statistical Analysis

In [ ]:
if results:
    # Compute statistics for each optimizer
    stats = {}
    
    for opt_name, opt_results in results.items():
        final_accs = [r['final_test_acc'] for r in opt_results]
        conv_epochs = [r['convergence_epoch'] for r in opt_results 
                       if r['convergence_epoch'] is not None]
        
        stats[opt_name] = {
            'final_acc_mean': np.mean(final_accs),
            'final_acc_std': np.std(final_accs),
            'conv_epoch_mean': np.mean(conv_epochs) if len(conv_epochs) > 0 else None,
            'conv_epoch_std': np.std(conv_epochs) if len(conv_epochs) > 0 else None,
            'conv_rate': len(conv_epochs) / len(opt_results),
        }
    
    # Display as table
    import pandas as pd
    
    df = pd.DataFrame(stats).T
    print("\nOptimizer Statistics:")
    print(df.to_string())

## Optimizer Behavior Analysis

In [ ]:
# Analyze learning dynamics
if results:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss curves
    for opt_name, opt_results in results.items():
        epochs = opt_results[0]['epochs']
        
        # Average over seeds
        train_losses_all = [r['train_losses'] for r in opt_results]
        mean_loss = np.mean(train_losses_all, axis=0)
        
        axes[0].plot(epochs, mean_loss, label=opt_name, linewidth=2)
    
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Training Loss')
    axes[0].set_title('Training Loss Curves')
    axes[0].legend()
    axes[0].set_yscale('log')
    axes[0].grid(True, alpha=0.3)
    
    # Train-test gap
    for opt_name, opt_results in results.items():
        epochs = opt_results[0]['epochs']
        
        train_accs = np.mean([r['train_accs'] for r in opt_results], axis=0)
        test_accs = np.mean([r['test_accs'] for r in opt_results], axis=0)
        
        gap = train_accs - test_accs
        
        axes[1].plot(epochs, gap, label=opt_name, linewidth=2)
    
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Train-Test Gap')
    axes[1].set_title('Generalization Gap')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## Export for Paper

Generate publication-quality figures.

In [ ]:
# Set publication style
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 16

# Generate figure
if results:
    fig = plot_optimizer_comparison(
        all_results=results,
        metric='test_accs',
        title='',  # Remove title for paper
    )
    
    # Save as PDF for LaTeX
    fig.savefig('../papers/track1_optimizer/figures/main_result.pdf', 
                dpi=300, bbox_inches='tight')
    print("Saved figure to papers/track1_optimizer/figures/main_result.pdf")